# Imports

## Standard Library Modules

In [ ]:
from datetime import datetime
import csv

## Third Party Modules

## Constants

### Global Constants

In [ ]:
import GlobalConstants

### Debug Level Flags

In [ ]:
#import DebugConstants as db

# Stuff to be Moved

## Testing

In [ ]:
# def test_instance_generation(num_tests):
#     assert not is_connected(nx.empty_graph(NUM_NODES))
#     assert is_connected(get_random_connected_graph())
#     G = get_random_connected_graph()
#     for i in range(num_tests):
#         request = MulticastRequest(MAX_MULTICAST_SIZE, G)
#         source = request.source
#         recipients = request.recipients
#         assert request.source is not None
#         assert request.recipients
#         assert not request.source in request.recipients
#         assert isinstance(request.recipients, set)
#     instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
#     for request in instance.requests:
#         assert request.source in instance.graph
#         assert request.recipients.issubset(instance.graph)

# def test_LP_creation(num_tests):
#     instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
#     reduced_LP = create_LP(instance.graph, instance.requests)
#     assert len(reduced_LP.getVars()) == 1
#     assert len(reduced_LP.getConstrs()) == NUM_EDGES + NUM_MULTICAST_REQUESTS, "Num of constraints: {} Expected: {}".format(len(reduced_LP.getConstrs()), NUM_EDGES + NUM_MULTICAST_REQUESTS)
    
# def test_subproblem(num_tests):
#     instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
#     reduced_LP = create_LP(instance.graph, instance.requests)
#     column_generator = Approx2MulticastPackingColumnGenerator(instance, reduced_LP)
    
#     new_trees = column_generator.generate_new_trees()
#     assert (len(new_trees) == instance.num_requests)
#     for tree in new_trees:
#         assert cost(tree, 1) == len(tree.edges()), "cost: {} edges: {}".format(cost(tree, 1), len(tree.edges()))
           
#     assert len(reduced_LP.getVars()) == 1+instance.num_requests
    
# def run_tests(num_tests):
#     test_instance_generation(num_tests)
#     test_LP_creation(num_tests)
#     test_subproblem(num_tests)

# NUM_TESTS = 0
# #run_tests(NUM_TESTS)

## Rounding Algorithms

### (Totally) Randomized Rounding

In [ ]:
def randRoundingHelper(fracSoln):
    randNum = random.random()
    for T in fracSoln:
        randNum -= fracSoln[T]
        if randNum < 0:
            return T
    
    
def randRounding(MCPsolver):
    intSoln = list()
    fracSoln = MCPsolver.solution[MCPsolver.iteration-1]
    for i in range(MCPsolver.instance.num_requests):
        intSoln.append(FrozenDict({randRoundingHelper(fracSoln[i]) : 1}))
        
    return tuple(intSoln)

# Run the solver

In [ ]:
from RunExpirement import runExpirement
datetime_str = datetime.now().strftime("%Y-%m-%d_%H_%M")
print(datetime_str)
tableHeaders = ["Label",
                "Solver", 
                "Block Apx Ratio", 
                "Vertices",
                "Edges",
                "Requests",
                "Group Size", 
                "Max Delay",
                "Congestion", 
                "Potential",
                "Iterations",
                "Total Time",
                "Avg. Time/It",
                "Stop Flags"
                ]
with open("outputs/{}.csv".format(datetime_str),'w') as allFile:
        allWriter = csv.writer(allFile)
        allWriter.writerow(tableHeaders)
        
with open("outputs/summaries/summary_{}.csv".format(datetime_str),'w') as sumFile:
        sumWriter = csv.writer(sumFile)
        sumWriter.writerow(tableHeaders)

In [ ]:
paramList = []
paramList += [("Requests",
               3*GlobalConstants.NUM_NODES, 
               3*GlobalConstants.NUM_EDGES, 
               i,
               3*GlobalConstants.MAX_MULTICAST_SIZE,
               3*GlobalConstants.DELAY
              ) for i in range(1, 26, 3)]
paramList += [("Group Size",
               3*GlobalConstants.NUM_NODES, 
               3*GlobalConstants.NUM_EDGES, 
               3*GlobalConstants.NUM_MULTICAST_REQUESTS, 
               i,
               3*GlobalConstants.DELAY
              ) for i in range(2, 3*GlobalConstants.NUM_NODES+1, 5)]

paramList += [("Instance size",
               2*i, # Nodes
               3*i, # Edges
               2*i, # Requests
                 i, # Group Size
                 0  # Delay
              ) for i in range(5, 51, 5) ]

numReps = 1

solverList = runExpirement(datetime_str, numReps, paramList, [1, 2], ["CG","PC"])

# End

In [ ]:
for solver in solverList:
    print(type(solver))
    print(solver.t)
    print("{:05b}".format(solver.stop_flag))